In [5]:
import numpy as np  
import pandas as pd 
import matplotlib.pyplot as plt 
%matplotlib inline 

In [6]:
import json 

from geopy.geocoders import Nominatim 

import requests 
from pandas.io.json import json_normalize 


import matplotlib.cm as cm
import matplotlib.colors as colors


from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances_argmin_min

import folium  

print('Libraries imported.')

Libraries imported.


In [7]:
import wget

url="https://infra.datos.gob.ar/catalog/modernizacion/dataset/7/distribution/7.29/download/localidades-censales.csv"
path="C:/Users/u605929/Documents/CURSOS/PYTHON/Ciencia de Datos Aplicada Programa Especializado/4. Capstone/Batalla de los Vecindarios/"
wget.download(url,path)

ModuleNotFoundError: ignored

In [ ]:
url_prov="https://infra.datos.gob.ar/catalog/modernizacion/dataset/7/distribution/7.7/download/provincias.csv"
wget.download(url_prov,path)

In [ ]:
url_internet="http://api.datosabiertos.enacom.gob.ar/api/v2/datastreams/CONEC-AL-SERVI-DE-INTER/data.csv?auth_key=44a38fbffd39c9f7d84e8e7dd2e1d02f0950e611&download=1"
wget.download(url_internet,path)

In [ ]:
localidades=pd.read_csv("localidades-censales.csv")
internet=pd.read_csv("ConectividadalserviciodeInternet.csv")

In [ ]:
provincias=pd.read_csv(path+"provincias.csv")

In [ ]:
regiones=pd.DataFrame(localidades.groupby(["provincia_nombre"]).size(),columns=['municipios']).sort_values(by="municipios")

# Initialize a Figure and an Axes
fig, ax = plt.subplots()

# Fig size
fig.set_size_inches(10,8)

# Create horizontal bars
ax.barh(y=regiones.index, width=regiones.municipios);

# Add title
ax.set_title('Cantidad de Municipios por Región en Argentina');

In [ ]:
regiones["Provincias"]=regiones.index

geo_prov=regiones.merge(provincias,left_on="Provincias",right_on="nombre")

In [ ]:
#url_map="https://dnsg.ign.gob.ar/apps/api/v1/capas-sig/Geodesia+y+demarcaci%C3%B3n/L%C3%ADmites/provincia/json"
url_map="https://infra.datos.gob.ar/catalog/modernizacion/dataset/7/distribution/7.12/download/provincias.geojson"
filename=wget.download(url_map)

print('GeoJSON file downloaded!')

In [ ]:
costa_atlan=geo_prov.loc[[1,2,5,15,23]]
costa_atlan=costa_atlan.drop(["categoria","nombre","fuente","id","iso_id","nombre_completo","Provincias"],axis=1)

In [ ]:
costa_atlan.reset_index(drop=True,inplace=True)
costa_atlan.rename(columns={"iso_nombre": "Provincia", "B": "c"},inplace=True)

In [ ]:
address = 'Argentina'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Argentina are {}, {}.'.format(latitude, longitude))

In [ ]:
# crear un mapa de Manhattan usando los valores de latitud y longitud
map_costa = folium.Map(location=[latitude, longitude], zoom_start=4)

# añadir los marcadores al mapa
for lat, lng, label in zip(costa_atlan['centroide_lat'], costa_atlan['centroide_lon'], costa_atlan['Provincia']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=8,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_costa)  
    
map_costa

In [ ]:
costa_atlan

In [ ]:
#localidades con acceso al mar fuente Wikipedia
loc_costa=pd.read_csv("Localidades_Costa.csv", encoding='latin-1')

#Se limpia los datos 
costa_limpio=pd.DataFrame(data=loc_costa["Localidades_Costa"].str.strip())

#Se hace merge con el repositorio de localidades
costa_merge=localidades.merge(costa_limpio,left_on='nombre',right_on='Localidades_Costa')

print("Localidades de la Costa",costa_limpio.shape)
print("Merge con el Repositorio",costa_merge.shape)

In [ ]:
#Se limpia las localidades que pueden tener nombre repetido y esten por fuera de las localidades en análisis.
costa_data=costa_merge[(costa_merge["provincia_nombre"]=="Buenos Aires")|
            (costa_merge["provincia_nombre"]=="Chubut")|
            (costa_merge["provincia_nombre"]=="Santa Cruz")|
            (costa_merge["provincia_nombre"]=="Río Negro")|
            (costa_merge["provincia_nombre"]=="Tierra del Fuego, Antártida e Islas del Atlántico Sur")
            ]

In [ ]:
CLIENT_ID = 'LM5HN3042JGPC50VLXUT2XG4L1Z4BN3DY1S5MBA5ANSVO54V' # su ID de Foursquare
CLIENT_SECRET = 'YCBLJNJX2WJMP0CKANFLTPXGSAQIARRGUKPB5BRROGAWPJO3' # Secreto de Foursquare
VERSION = '20180605' # versión de la API de Foursquare
LIMIT = 100 # Un valor límite para la API de Foursquare

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [ ]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # crear la URL de solicitud de API
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # solicitud GET
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # regresa solo información relevante de cada sitio cercano
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighbourhood', 
                  'Neighbourhood Latitude', 
                  'Neighbourhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [ ]:
cos_venues = getNearbyVenues(names=costa_data['nombre'],
                                   latitudes=costa_data['centroide_lat'],
                                   longitudes=costa_data['centroide_lon']
                                  )

In [ ]:
  cos_venues.groupby('Neighbourhood').count()

In [ ]:
print('Hay {} categorias únicas.'.format(len(cos_venues['Venue Category'].unique())))

In [ ]:
# codificación
costa_onehot = pd.get_dummies(cos_venues[['Venue Category']], prefix="", prefix_sep="")

# añadir la columna de barrio de regreso al dataframe
costa_onehot['Neighbourhood'] = cos_venues['Neighbourhood'] 

# mover la columna de barrio a la primer columna
fixed_columns = [costa_onehot.columns[-1]] + list(costa_onehot.columns[:-1])
costa_onehot = costa_onehot[fixed_columns]

#Agrupación por el promedio
costa_grouped = costa_onehot.groupby('Neighbourhood').mean().reset_index()

In [ ]:
costa_grouped.head()

In [ ]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [ ]:
num_top_venues = 3

indicators = ['st', 'nd', 'rd']

# crear las columnas acorde al numero de sitios populares
columns = ['Neighbourhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# crear un nuevo dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighbourhood'] = costa_grouped['Neighbourhood']

for ind in np.arange(costa_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(costa_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

In [ ]:
##Sumar variables de Internet
acceso=internet.drop(["Partido","link","Latitud","Longitud"],axis=1)

#Se limpia las localidades que pueden tener nombre repetido y esten por fuera de las localidades en análisis.
int_var=acceso[(acceso["Provincia"]=="BUENOS AIRES")|
            (acceso["Provincia"]=="CHUBUT")|
            (acceso["Provincia"]=="SANTA CRUZ")|
            (acceso["Provincia"]=="RIO NEGRO")|
            (acceso["Provincia"]=="TIERRA DEL FUEGO")
            ]

In [ ]:
#Se hace un merge para trabajar solo con las localidades en estudio
variables_data=costa_grouped[["Neighbourhood"]].merge(int_var,how="left",left_on="Neighbourhood",right_on="Localidad")

In [ ]:
variables_data.columns

In [ ]:
variables_data.head()

In [ ]:
# example of a ordinal encoding
from numpy import asarray
from sklearn.preprocessing import OrdinalEncoder
#define data
data = variables_data[['ADSL','CABLEMODEM','DIALUP','FIBRAOPTICA','SATELITAL','WIRELESS', 'TELEFONIAFIJA','3G','4G']]
# define ordinal encoding
encoder = OrdinalEncoder()
# transform data
result = encoder.fit_transform(data)

In [ ]:
#Convertir a un dataframe
internet_grouped=pd.DataFrame(data=result,columns=[data.columns]).reset_index(drop=True)

#Concatenar con las variables geograficas
var_grouped=pd.concat([costa_grouped,internet_grouped], axis=1)

In [ ]:
internet_grouped.describe()

In [ ]:
#Seleccion de Columnas con la información por tipo de red de acceso
internet_data=var_grouped.loc[:,var_grouped.columns[[0] +list(range(76, var_grouped.shape[1]))]]

#Argupaciones
internet_data["Red_Movil"]=np.where(internet_data.iloc[:,8]+internet_data.iloc[:,9]>0,"SI","NO")
internet_data["Red_BandaAncha"]=np.where(internet_data.iloc[:,1]+internet_data.iloc[:,2]+internet_data.iloc[:,4]>0,"SI","NO")
internet_data["Red_BandaAngosta"]=np.where(internet_data.iloc[:,3]+internet_data.iloc[:,5]+internet_data.iloc[:,6]>0,"SI","NO")
internet_data["Telefono_Fijo"]=np.where(internet_data.iloc[:,7]>0,"SI","NO")

#Se dejan solo las columnas agrupadas y se trae la informeación de Población
acceso_data=internet_data.loc[:,internet_data.columns[[0] + list(range(10,internet_data.shape[1]))]]
acceso_data.insert(1,'Poblacion', variables_data["Poblacion"])
acceso_data.head()

In [ ]:
#Variables para el modelo sin el campo Neighbourhood
costa_grouped_clustering = var_grouped.drop('Neighbourhood', 1)

In [ ]:
#Curva del Codo
X=costa_grouped_clustering
Nc = range(1, 20)
kmeans = [KMeans(n_clusters=i) for i in Nc]
kmeans
score = [kmeans[i].fit(X).score(X) for i in range(len(kmeans))]
score
plt.plot(Nc,score)
plt.xlabel('Number of Clusters')
plt.ylabel('Score')
plt.title('Elbow Curve')
plt.show()

In [ ]:
# establecer el número de agrupaciones
kclusters = 3

# ejecutar k-means
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(costa_grouped_clustering)

# revisar las etiquetas de las agrupaciones generadas para cada fila del dataframe
kmeans.labels_[0:10] 

In [ ]:
costa_localidades=costa_data[["Localidades_Costa","provincia_nombre","centroide_lat","centroide_lon"]].merge(acceso_data,left_on='Localidades_Costa', right_on='Neighbourhood')
costa_localidades=costa_localidades.drop("Neighbourhood",axis=1)
costa_localidades.head()

In [ ]:
# añadir etiquetas
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

# juntar costa_grouped con costa_data 
costa_merged = costa_localidades.merge(neighborhoods_venues_sorted,left_on='Localidades_Costa', right_on='Neighbourhood')

costa_merged.head() # revisar las ultimas columnas

In [ ]:
# crear mapa
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=4)

# establecer el esquema de color para las agrupaciones
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# añadir marcadores al mapa
markers_colors = []
for lat, lon, poi, cluster in zip(costa_merged['centroide_lat'], costa_merged['centroide_lon'], costa_merged['Localidades_Costa'], costa_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=10,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [ ]:
costa_merged.loc[costa_merged['Cluster Labels'] == 0, costa_merged.columns[[1] + list(range(4, costa_merged.shape[1]))]].sort_values("Poblacion",ascending=False)

In [ ]:
costa_merged.loc[costa_merged['Cluster Labels'] == 1, costa_merged.columns[[1] + list(range(4, costa_merged.shape[1]))]].sort_values("Poblacion",ascending=False)

In [ ]:
costa_merged.loc[costa_merged['Cluster Labels'] == 2, costa_merged.columns[[1] + list(range(4, costa_merged.shape[1]))]].sort_values("Poblacion",ascending=False)